In [1]:
import pandas as pd

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_38232/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [49]:
import requests
base_url = "http://localhost:7999"

## NSE Tickers Wrangling

In [13]:
instruments = pd.read_csv("data/upstox/NSE.csv")
instruments

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
...,...,...,...,...,...,...,...,...,...,...,...,...
46188,NCD_FO|15778,15778.0,USDINR2461484.375PE,NaN,NaN,2024-06-14,84.375,0.25,1.0,OPTCUR,PE,NCD_FO
46189,NCD_FO|15779,15779.0,USDINR2461489.5CE,NaN,NaN,2024-06-14,89.500,0.25,1.0,OPTCUR,CE,NCD_FO
46190,NCD_FO|15780,15780.0,USDINR2461489.5PE,NaN,NaN,2024-06-14,89.500,0.25,1.0,OPTCUR,PE,NCD_FO
46191,NCD_FO|15781,15781.0,USDINR2461489.75CE,NaN,NaN,2024-06-14,89.750,0.25,1.0,OPTCUR,CE,NCD_FO


In [41]:
instruments_eq = instruments.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
instruments_eq

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
...,...,...,...,...,...,...,...,...,...,...,...,...
6635,NSE_EQ|INE358U01012,11394.0,ZOTA,ZOTA HEALTH CARE LIMITED,485.00,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6636,NSE_EQ|INE840M01016,29050.0,ZUARI,ZUARI AGRO CHEMICALS LTD,199.90,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6637,NSE_EQ|INE217A01012,3827.0,ZUARIIND,ZUARI INDUSTRIES LIMITED,351.75,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6638,NSE_EQ|INE010B01027,7929.0,ZYDUSLIFE,ZYDUS LIFESCIENCES LTD,947.40,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [42]:
instruments_index = instruments.query('instrument_type == "INDEX" and exchange == "NSE_INDEX"')
instruments_index

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
...,...,...,...,...,...,...,...,...,...,...,...,...
72,NSE_INDEX|Nifty Tata 25 Cap,NaN,NaN,Nifty Tata 25 Cap,16019.90,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
73,NSE_INDEX|Nifty Multi Mfg,NaN,NaN,Nifty Multi Mfg,12992.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
74,NSE_INDEX|NIFTY OIL AND GAS,NaN,NaN,NIFTY OIL AND GAS,11635.30,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
75,NSE_INDEX|Nifty MidSml Hlth,NaN,NaN,Nifty MidSml Hlth,32965.90,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


GOAL ~ 2400 NSE listed companies
Remove:
- Government issued instruments
- Anything with %
- NSETEST

In [59]:
goi_issued_eq = ["GOI STRIPS", "GOI TBILL", "GOI LOAN"]

def remove_names(name, remove_list):
    return all(i not in name for i in remove_list)

x = instruments_eq[instruments_eq["name"].apply(lambda x: remove_names(x, goi_issued_eq))]

randoms = ["%", "NSETEST"]

y = x[x["name"].apply(lambda x: all(i not in x for i in randoms))]
y.shape


(2894, 12)

## Internal API

POST Equities to BE

In [60]:
s, f = 0, 0

rename_cols = {
    "tradingsymbol": "ticker",
    "instrument_key": "upstox_instrument_key"
}
instruments_eq.rename(columns=rename_cols, inplace=True)

cols = ["exchange", "exchange_token", "name", "ticker", "instrument_type", "upstox_instrument_key", "lot_size"]
payload = instruments_eq[cols].to_dict(orient="records")

for p in payload:
    p["exchange"] = p["exchange"].split("_")[0]

    resp = requests.post(base_url + "/tickers/", json=p )
    if resp.status_code == 201:
        s += 1
    else:
        f += 1
        print(resp.json())

print(f"Success: {s}, Failure: {f}")


/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_38232/4276814953.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instruments_eq.rename(columns=rename_cols, inplace=True)


Success: 6563, Failure: 0


Fetch and POST market data for set day range:

Past eight years, monthly candle sticks.

## External API

In [63]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|IN002023Y458/day/2024-01-05/2024-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"status":"success","data":{"candles":[]}}
